In [ ]:
# 동적 크롤링
# %pip install selenium

# 드라이버 자동 관리
# %pip install webdriver-manager

In [ ]:
from selenium import webdriver   # 웹 브라우저 자동 제어
from selenium.webdriver.common.by import By  # HTML 요소를 찾을 때 기준을 지정
from selenium.webdriver.chrome.options import Options  # 크롬 드라이버 옵션 설정
import pandas as pd
import time
import os
# import random

from selenium.webdriver.support.ui import WebDriverWait  # 특정 요소가 나타날 때까지 기다림
from selenium.webdriver.support import expected_conditions as EC  # 기다리는 조건을 정의

In [ ]:
# 크롬 드라이버 옵션 설정
options = Options()
options.add_argument("--headless")  # 창 안 띄우고 실행

# driver : 크롬 브라우저 객체
driver = webdriver.Chrome(options=options)

# 상품 ID (무신사 상품번호)
goods_no = "912314"

### 1. 상품 상세 페이지에서, 상품 정보 크롤링
- 상품명, 가격, 할인율, 후기 수, 조회수, 누적 판매량

In [ ]:
product_url = f"https://www.musinsa.com/products/{goods_no}"
driver.get(product_url)
time.sleep(2)

# By.XPATH : HTML 구조를 따라 논리적으로 찾기
# "//span[contains(@class, 'GoodsName')]" 
# : 페이지 안의 모든 <span> 태그 중에서, class 속성 안에 'GoodsName'이라는 문자열이 포함된 태그를 찾아라

# 상품명
try:
    product_name = driver.find_element(By.XPATH, "//span[contains(@class, 'GoodsName')]").text.strip()
except:
    product_name = None

# 가격
try:
    price_text = driver.find_element(
        By.XPATH, "//span[contains(@class, 'Price__CalculatedPrice')]"
    ).text.strip()
except:
    price_text = None

# 할인율
try:
    discount_text = driver.find_element(
        By.XPATH, "//span[contains(@class, 'Price__DiscountRate')]"
    ).text.strip()
except:
    discount_text = None

# 후기 수
try:
    review_count_text = driver.find_element(
        By.XPATH, "//span[contains(@class, 'underline') and contains(text(),'후기')]"
    ).text.strip()
except:
    review_count_text = None

# 스크롤 필요 항목 - 조회수, 누적 판매량
driver.execute_script("window.scrollTo(0, 500)")  # 아래로 살짝
time.sleep(1.5)

# 조회수
try:
    # <dt> 안의 텍스트 내용 중 '조회'라는 글자가 포함된 태그를 찾아서,
    # 그 태그(dt) 바로 뒤에 나오는 형제 요소 중 <dd> 태그를 선택하겠다!
    view_text = driver.find_element(By.XPATH, "//dt[contains(text(), '조회')]/following-sibling::dd").text
except:
    view_text = None
    
# 누적 판매량
try:
    sales_text = driver.find_element(By.XPATH, "//dt[contains(text(), '누적판매')]/following-sibling::dd").text
except:
    sales_text = None


import re

# '6만개 이상', '7만회 이상', '12,543회' 등 다양한 형식을 int로 변환
def extract_korean_number(text): 
    if not text: 
        return None
    
    text = text.replace(",", "")  # 콤마 제거
    
    # '6만' or '7만회 이상' 같은 패턴 처리
    match = re.search(r"(\d+)(\.\d+)?\s*만", text)
    if match:
        num = float(match.group(1))
        return int(num * 10000)
    
    # 일반 숫자만 있는 경우
    digits = re.findall(r"\d+", text)
    if digits:
        return int(''.join(digits))
    
    return None


view_count = extract_korean_number(view_text)
sales_count = extract_korean_number(sales_text)
price = extract_korean_number(price_text)
discount_pct = extract_korean_number(discount_text)
review_count = extract_korean_number(review_count_text)


print("✅ 상품 정보 크롤링 완료")
print(f"상품명: {product_name}, 가격: {price}, 할인율(%): {discount_pct}")
print(f"리뷰 수: {review_count}, 조회수: 약 {view_count} 회 이상, 누적판매: 약 {sales_count} 개 이상")

### 2. 리뷰 더보기 페이지에서, 리뷰 데이터 크롤링
- 닉네임, 텍스트 리뷰, 별점

In [ ]:
options = Options()
# 창 띄우기
# options.add_argument("--headless")

driver = webdriver.Chrome(options=options)

# 리뷰 페이지 접근 - 최신순 정렬 전
review_url = f"https://www.musinsa.com/review/goods/{goods_no}?gf=A"
driver.get(review_url)
time.sleep(2)

# 스크롤 - '유용한순' 보일 정도
driver.execute_script("window.scrollTo(0, 650)")
time.sleep(0.5)

# '유용한순' span 클릭
try:
    useful_span = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), '유용한순')]"))
    )
    driver.execute_script("arguments[0].click();", useful_span)
    time.sleep(1)
    print("✅ 유용한순 클릭 완료")
except Exception as e:
    print("⚠️ 유용한순 클릭 실패:", e)

# 드롭다운에서 '최신순' span 클릭
try:
    recent_span = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), '최신순')]"))
    )
    driver.execute_script("arguments[0].click();", recent_span)
    time.sleep(2)
    print("✅ 최신순 정렬 완료")
except Exception as e:
    print("⚠️ 최신순 클릭 실패:", e)

In [ ]:
target_count = 5000  # 가져올 리뷰 수
collected = []
seen_reviews = set()

while len(collected) < target_count:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    review_boxes = driver.find_elements(
        By.CSS_SELECTOR, "div.review-list-item__Container-sc-15m8pom-0"
    )

    for box in review_boxes:
        try:
            review_text = box.find_element(
                By.CSS_SELECTOR,
                "span.text-body_13px_reg.w-full.text-black.font-pretendard"
            ).text.strip()
        except:
            review_text = None

        if not review_text or review_text in seen_reviews:
            continue  # 이미 본 리뷰는 스킵

        try:
            nickname = box.find_element(
                By.XPATH, ".//span[contains(@class,'UserProfileSection__Nickname')]"
            ).text.strip()
        except:
            nickname = None

        try:
            star_text = box.find_element(
                By.CSS_SELECTOR,
                "span.text-body_13px_semi.font-pretendard"
            ).text.strip()
            star = int(star_text) if star_text.isdigit() else None
        except:
            star = None

        collected.append((nickname, review_text, star))
        seen_reviews.add(review_text)

    print(f"누적 리뷰 개수: {len(collected)}")

    # 스크롤 끝 감지
    if len(collected) >= target_count:
        break


# DataFrame 생성
df = pd.DataFrame(collected, columns=["nickname", "review", "star"])

# 긴 문자열 전부 출력
pd.set_option("display.max_colwidth", None) 


driver.quit()

In [ ]:
df.head(20)

In [ ]:
df.shape

In [ ]:
df_add = df.copy()
df_add.shape

In [ ]:
df["product_name"] = product_name
df["price"] = price
df["discount_pct"] = discount_pct
df["review_count"] = review_count
df["view_count"] = view_count
df["sales_count"] = sales_count

df.head(10)

In [ ]:
df_add["product_name"] = product_name
df_add["price"] = price
df_add["discount_pct"] = discount_pct
df_add["review_count"] = review_count
df_add["view_count"] = view_count
df_add["sales_count"] = sales_count

df_add.head(10)

In [ ]:
# csv로 저장
path_folder = "../data/raw/review"

if not os.path.exists(path_folder):
    os.makedirs(path_folder)

df_add.to_csv(f"{path_folder}/musinsa_reviews_{goods_no}.csv", index=False, encoding="utf-8-sig")